In [1]:
// 这是不支持LCEL的，所以定制会比较麻烦。但是可以方便地实现记忆对话。
import {deepSeekChatModel} from './models/index.ts'
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";

const memory = new BufferMemory();
const chain = new ConversationChain({ llm: deepSeekChatModel, memory: memory });
const res1 = await chain.call({ input: "我是小明" });
res1

{
  response: "你好，小明！很高兴认识你。你今天过得怎么样？有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，我都很乐意为你提供帮助。😊"
}

In [2]:
const res2 = await chain.call({ input: "我叫什么？" });
res2

{ response: "你刚才告诉我你叫小明呀！😊 如果你有其他名字或者昵称，也可以告诉我哦！" }

内置Memory的机制  
首先是 BufferWindowMemory：


In [3]:
// 导入BufferWindowMemory
// 这里非常好理解，就是对聊天记录加了一个滑动窗口，只会记忆 k 个对话
// 可以节省token
import { BufferWindowMemory } from "@langchain/core/memory";
const memory = new BufferWindowMemory({ k: 1 });
const chain = new ConversationChain({ llm: model, memory: memory });

TypeError: BufferWindowMemory is not a constructor

ConversationSummaryMemory,官网提供的，随着聊天不断生成聊天记录摘要

In [ ]:
import { ConversationSummaryMemory } from "langchain/memory";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatDeepSeek } from "@langchain/deepseek";
const memory = new ConversationSummaryMemory({
    memoryKey: "summary",
    llm: new ChatDeepSeek({
        model: "deepseek-chat", 
        verbose: true,
    }),
  });

const model = deepSeekChatModel;
const prompt = PromptTemplate.fromTemplate(`
你是一个乐于助人的助手。尽你所能回答所有问题。

这是聊天记录的摘要:
{summary}
Human: {input}
AI:`);
const chain = new ConversationChain({ llm: model, prompt, memory, verbose: true });

const res1 = await chain.call({ input: "我是小明" });
const res2 = await chain.call({ input: "我叫什么？" });


将 BufferWindowMemory 和 ConversationSummaryMemory 结合起来，根据 token 数量，如果上下文历史过大时就切换到 summary，如果上下文比较小时就使用原始的聊天记录，也就成了 ConversationSummaryBufferMemory。

In [ ]:

import { ConversationSummaryBufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";
import {ChatDeepSeek} from '@langchain/deepseek'

// 目前不支持deepseek模型的token数计算
const model = deepSeekChatModel;
const memory = new ConversationSummaryBufferMemory({
  llm: new ChatDeepSeek({
        model: "deepseek-chat", 
        verbose: true,
    }),
  maxTokenLimit: 200
});
const chain = new ConversationChain({ llm: model, memory: memory, verbose: true });
const res1 = await chain.call({ input: "我是小明" });
console.log({res1});
const res2 = await chain.call({ input: "我叫什么？" });
console.log({res2});

[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我是小明",
  "history": ""
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatDeepSeek] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: 我是小明\nAI:",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:chain:ConversationChain > 2:llm:ChatDeepSeek] [10.01s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "你好，小明！很高兴认识你。你今天过得怎么样？有什么我可以帮忙的吗？无论是学习、工作还是生活中的问

Failed to calculate number of tokens, falling back to approximate count Error: Unknown model
    at getEncodingNameForModel (file:///C:/home/Projects/AI/ai-agents/node_modules/js-tiktoken/dist/chunk-Z5MDQTGX.js:247:13)
    at encodingForModel (file:///C:/home/Projects/AI/ai-agents/node_modules/@langchain/core/dist/utils/tiktoken.js:19:24)
    at ChatDeepSeek.getNumTokens (file:///C:/home/Projects/AI/ai-agents/node_modules/@langchain/core/dist/language_models/base.js:197:40)
    at ConversationSummaryBufferMemory.prune (file:///C:/home/Projects/AI/ai-agents/node_modules/langchain/dist/memory/summary_buffer.js:114:47)
    at eventLoopTick (ext:core/01_core.js:177:7)
    at async ConversationSummaryBufferMemory.saveContext (file:///C:/home/Projects/AI/ai-agents/node_modules/langchain/dist/memory/summary_buffer.js:96:9)
    at async ConversationChain.invoke (file:///C:/home/Projects/AI/ai-agents/node_modules/langchain/dist/chains/base.js:71:13)
    at async <anonymous>:17:14


[chain/end] [1:chain:ConversationChain] [10.03s] Exiting Chain run with output: {
  "response": "你好，小明！很高兴认识你。你今天过得怎么样？有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，我都很乐意为你提供帮助哦！😊"
}
{
  res1: {
    response: "你好，小明！很高兴认识你。你今天过得怎么样？有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，我都很乐意为你提供帮助哦！😊"
  }
}
[chain/start] [1:chain:ConversationChain] Entering Chain run with input: {
  "input": "我叫什么？",
  "history": "Human: 我是小明\nAI: 你好，小明！很高兴认识你。你今天过得怎么样？有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，我都很乐意为你提供帮助哦！😊"
}
[llm/start] [1:chain:ConversationChain > 2:llm:ChatDeepSeek] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not kno

Failed to calculate number of tokens, falling back to approximate count Error: Unknown model
    at getEncodingNameForModel (file:///C:/home/Projects/AI/ai-agents/node_modules/js-tiktoken/dist/chunk-Z5MDQTGX.js:247:13)
    at encodingForModel (file:///C:/home/Projects/AI/ai-agents/node_modules/@langchain/core/dist/utils/tiktoken.js:19:24)
    at ChatDeepSeek.getNumTokens (file:///C:/home/Projects/AI/ai-agents/node_modules/@langchain/core/dist/language_models/base.js:197:40)
    at ConversationSummaryBufferMemory.prune (file:///C:/home/Projects/AI/ai-agents/node_modules/langchain/dist/memory/summary_buffer.js:114:47)
    at eventLoopTick (ext:core/01_core.js:177:7)
    at async ConversationSummaryBufferMemory.saveContext (file:///C:/home/Projects/AI/ai-agents/node_modules/langchain/dist/memory/summary_buffer.js:96:9)
    at async ConversationChain.invoke (file:///C:/home/Projects/AI/ai-agents/node_modules/langchain/dist/chains/base.js:71:13)
    at async <anonymous>:23:14


[chain/end] [1:chain:ConversationChain] [10.85s] Exiting Chain run with output: {
  "response": "你刚刚告诉我你叫小明呀！😊 如果你有其他名字或者昵称，也可以告诉我哦！"
}
{ res2: { response: "你刚刚告诉我你叫小明呀！😊 如果你有其他名字或者昵称，也可以告诉我哦！" } }


 EntityMemory

在人类聊天的过程中，我们实际在建立的是对各种实体（Entity）的记忆，例如两个刚认识的人，我们聊职业、聊公司、聊餐馆，我们记忆中存储方式可能是根据实体进行分类存储，这个人是什么职业、年龄；这个公司是什么情况；餐馆是什么环境和味道。EntityMemory 希望模拟的就是在聊天中去生成和更新不同的实体的描述。

In [10]:

import { EntityMemory, ENTITY_MEMORY_CONVERSATION_TEMPLATE } from "langchain/memory";
import { ConversationChain } from "langchain/chains";

const model = deepSeekChatModel;
const memory = new EntityMemory({
    llm: new ChatDeepSeek({
        model: "deepseek-chat",
        verbose: true 
    }),
    chatHistoryKey: "history",
    entitiesKey: "entities"
});
const chain = new ConversationChain({ 
    llm: model, 
    prompt: ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory: memory, 
    verbose: true 
});

const res1 = await chain.call({ input: "我是小明" });
console.log({res1});
const res2 = await chain.call({ input: "我叫什么？" });
console.log({res2});

[llm/start] [1:llm:ChatDeepSeek] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: my name 